In [21]:
import json
import urllib.request

import requests
import os
import env

def getMetaCards(card_ids):
    responses = []
    for card_id in card_ids:
        url = "http://metabase.jabama.com/api/card/" + card_id + "/query"

        headers = {
            'X-Metabase-Session': "daabf2cf-c6f5-413f-9cb3-e1bffad2d699",
            'Cache-Control': "no-cache",
            'Postman-Token': "4914830f-7ad3-4213-8738-1133e80d1c9b"
            }

        responses.append(requests.request("POST", url, headers=headers))    
    return responses

def getDataFromResponses(responses):
    responses_data = []
    for response in responses:
        cols = json.loads(response.text)['data']['cols']
        rows = json.loads(response.text)['data']['rows']

        col_titles = []
        for col in cols:
            col_titles.append(col['name'])

        responses_data.append({
            "columns": col_titles,
            "rows": rows
        })
    return responses_data

def formatMessageBlocks(response_tables):
    message_blocks = []
    for response_table in response_tables:
        plain_text = ""
        response_table.get("columns").pop(0)
        for row in response_table.get("rows"):
            plain_text = plain_text + "*" + str(row[0]) + "*\n"
            indx = 1
            for col in response_table.get("columns"):
                plain_text = plain_text + col + ": " + str(row[indx]) + "\n"
                indx = indx + 1
            plain_text = plain_text + "\n"
        message_blocks.append({
            "type": "section",
            "text": {
                "type": "mrkdwn",
                "text": plain_text
            }
        })
        message_blocks.append({
            "type": "divider"
        })
    return message_blocks

def getUserIDByEmail(emails):
    responses = []
    for email in emails:  
        querystring = {
            "email" : email
        }
        headers = {
            'Content-Type': "application/json",
            'Authorization': "Bearer " + env.METABOT_TOKEN,
            'User-Agent': "PostmanRuntime/7.20.1",
            'Accept': "*/*",
            'Cache-Control': "no-cache",
            'Postman-Token': "1eed9ef6-4ab9-44d9-bf2a-650df7574c84,7e6fc185-e1fd-4ca8-9806-782d0de54a84",
            'Host': "slack.com",
            'Accept-Encoding': "gzip, deflate",
            'Connection': "keep-alive",
            'cache-control': "no-cache"
            }
        response = requests.request("GET", env.SLACK_API_LOOK_UP_BY_EMAIL, headers=headers, params=querystring)
        responses.append(json.loads(response.text)['user']['id'])
    return responses
        
def sendMessagesToSlackUsers(user_ids, message_blocks):
    responses = []
    for user_id in user_ids:
        msg_indx = 0
        payload = {
        "channel" : user_id,
        "blocks": message_blocks,
        "as_user": True
        }
        headers = {
            'Content-Type': "application/json",
            'Authorization': "Bearer " + env.METABOT_TOKEN,
            'Cache-Control': "no-cache",
            'Postman-Token': "3dea07a8-cf24-47cb-b8cd-2a141a41a0b3"
            }
        response = requests.request("POST", env.SLACK_API_POST_MESSAGE, data=str(payload), headers=headers)
        responses.append({
            'user_id': user_id,
            'message_index': msg_indx,
            'status': json.loads(response.text)['ok']
        })
        msg_indx = msg_indx + 1
    return responses

metacard_responses = getMetaCards([env.CARD_ID_ACCOMMODATION, env.CARD_ID_TOTAL])
responses_data = getDataFromResponses(metacard_responses)
message_blocks = formatMessageBlocks(responses_data)

user_ids = getUserIDByEmail([env.EMAIL_NEGAR, env.EMAIL_HOSSEIN])
send_messages_status = sendMessagesToSlackUsers(user_ids, message_blocks)

print(send_messages_status)

[{'user_id': 'UR6RJUHGX', 'message_index': 0, 'status': True}, {'user_id': 'UPFF8H2HY', 'message_index': 0, 'status': True}]
